In [67]:
import pandas as pd

In [68]:
client60 = pd.read_csv("../log-single-client60/nginx-minis2.csv")
client60 = client60.select_dtypes(include=['int', 'float'])
client60_h2_start = 1675340392 + 30
client60_h2_end = 1675342353 - 30
client60_h3_start = 1675342960 + 30
client60_h3_end = 1675345284 -30
client60.dtypes

Time         int64
UID          int64
PID          int64
%usr       float64
%system    float64
%guest     float64
%wait      float64
%CPU       float64
CPU          int64
dtype: object

In [69]:
def cpu_performance(df: pd.DataFrame, start: int, end: int):
    df_groupby = df[(start <= df["Time"]) & (df["Time"] <= end)].groupby("Time")
    df_groupby = df_groupby.sum()
    return df_groupby


In [70]:
client60_h2 = cpu_performance(client60, client60_h2_start, client60_h2_end)
client60_h2[[r"%usr", r"%system", r"%CPU"]].describe()

,%usr,%system,%CPU
count,1902.000000,1902.000000,1902.000000
mean,11.062319,5.000131,16.062450
std,3.564572,1.972240,4.794644
min,1.000000,0.000000,1.000000
25%,9.000000,4.000000,13.000000
50%,11.000000,5.000000,16.000000
75%,13.000000,6.000000,19.000000
max,27.000000,15.840000,39.000000


In [71]:
client60_h3 = cpu_performance(client60, client60_h3_start, client60_h3_end)
client60_h3[[r"%usr", r"%system", r"%CPU"]].describe()

,%usr,%system,%CPU
count,2264.000000,2264.000000,2264.000000
mean,28.935499,29.163366,58.098865
std,3.727589,3.704260,5.226097
min,15.000000,14.000000,39.000000
25%,26.932500,27.000000,55.000000
50%,29.000000,29.000000,58.000000
75%,31.000000,32.000000,61.000000
max,50.000000,45.000000,84.000000
